## Saving pydantic-validated data to SQLite database with SQLModel

Reference: 
1. Create a SQLModel Model https://sqlmodel.tiangolo.com/#create-a-sqlmodel-model & https://sqlmodel.tiangolo.com/#sqlalchemy-and-pydantic

In [2]:
%pip install requests
%pip install lxml
%pip install pandas
%pip install watermark
%pip install pydantic==2.3.0
%pip install pydantic-core==2.6.3
%pip install sqlmodel
%pip install word2number

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 1

#### Scrape the website content

In [3]:
from pydantic import (BaseModel, 
                      validate_call,
                      computed_field, 
                      field_serializer,
                      HttpUrl,
                      )
from sqlmodel import Field, SQLModel, Session, create_engine

class BookModel(SQLModel, table=True):
  prices: list[str]
  book_titles: list[str]
  book_urls: list[str]
  star_ratings_classes: list[str]

  @computed_field
  @property
  def star_ratings(self) -> list[str]:
    return [star_rating.replace("star-rating ","") for star_rating in self.star_ratings_classes]

  @field_serializer('star_ratings')
  def serialize_star_rating(star_ratings: list[str]) -> list[int]:
    return [w2n.word_to_num(star_rating) for star_rating in star_ratings]

engine = create_engine("sqlite:///database.db")
SQLModel.metadata.create_all(engine)

ValueError: <class 'list'> has no matching SQLAlchemy type

In [3]:
%%time

import requests
from lxml import html
from word2number import w2n # convert number words (eg. twenty one) to numeric digits (21)
import pandas
from typing import Union # add type hint

class BookSpider:
    def __init__(self):
        self.base_url: AnyHttpUrl = "https://books.toscrape.com"
        self.session: requests.sessions.Session  = requests.Session()
        self.headers: dict[str, str] = {
            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'
            }
        self.bookmodel = BookModel

    def parse(self, page_num: int=50) -> None:
        results = []
        # loop the page number to scrape
        for page_num in range(1,page_num):
            url = self.base_url + f"/catalogue/page-{page_num}.html"
            output_data = self._fetch_data(url=url)
            validated_bookmodel_data = self._validate_bookmodel_data(output_data)
            results.append(validated_bookmodel_data)
        return results

    @validate_call(validate_return=True)
    def _fetch_data(self, url: HttpUrl) -> dict[str, list]:
        response = self.session.request("GET", url)
        tree = html.fromstring(response.content)      
        prices = tree.xpath("//article[@class='product_pod']//div[@class='product_price']\
            /p[@class='price_color']/text()")
        # bug: how to break down prices -> prices & currencies        
        book_titles = tree.xpath("//article[@class='product_pod']/h3/a/text()")
        book_urls = tree.xpath("//article[@class='product_pod']/h3/a/@href")
        star_ratings_classes = tree.xpath("//article[@class='product_pod']/p/@class")

        data_list = {
                    "prices": prices,
                    "star_ratings_classes": star_ratings_classes,
                    "book_urls": book_urls, 
                    "book_titles": book_titles,
                }
        return data_list

    @validate_call(validate_return=True)
    def _validate_bookmodel_data(self, books_data: dict[str, list]) -> dict[str, list]:
        # validate book data with pydantic
        validated_bookmodel_data = dict(self.bookmodel.model_validate(books_data).model_dump()) ## model_dump is to compute field
        return validated_bookmodel_data

book_spider = BookSpider()
data_list = book_spider.parse()

CPU times: user 660 ms, sys: 406 ms, total: 1.07 s
Wall time: 4.54 s


In [4]:
import pandas

df = pandas.DataFrame(data_list)
df.explode(['prices', 'book_titles', 'book_urls', 'star_ratings_classes', 'star_ratings'])

,prices,book_titles,book_urls,star_ratings_classes,star_ratings
0,£51.77,A Light in the ...,a-light-in-the-attic_1000/index.html,star-rating Three,3
0,£53.74,Tipping the Velvet,tipping-the-velvet_999/index.html,star-rating One,1
0,£50.10,Soumission,soumission_998/index.html,star-rating One,1
0,£47.82,Sharp Objects,sharp-objects_997/index.html,star-rating Four,4
0,£54.23,Sapiens: A Brief History ...,sapiens-a-brief-history-of-humankind_996/index...,star-rating Five,5
...,...,...,...,...,...
48,£40.44,Icing (Aces Hockey #2),icing-aces-hockey-2_25/index.html,star-rating Four,4
48,£45.24,"Hawkeye, Vol. 1: My ...",hawkeye-vol-1-my-life-as-a-weapon-hawkeye-1_24...,star-rating Three,3
48,£34.96,Having the Barbarian's Baby ...,having-the-barbarians-baby-ice-planet-barbaria...,star-rating Four,4
48,£56.76,"Giant Days, Vol. 1 ...",giant-days-vol-1-giant-days-1-4_22/index.html,star-rating Four,4


## Computing environment

In [5]:
%load_ext watermark

%watermark

# print out pypi packages used
%watermark --iversions

# date
%watermark -u -n -t -z

Last updated: 2024-03-10T10:08:23.577980+00:00

Python implementation: CPython
Python version       : 3.10.12
IPython version      : 8.22.2

Compiler    : GCC 11.4.0
OS          : Linux
Release     : 6.1.75-060175-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 16
Architecture: 64bit

word2number: 1.1
pandas     : 2.2.1
requests   : 2.31.0
lxml       : 5.1.0

Last updated: Sun Mar 10 2024 10:08:23UTC

